In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 32 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   project           236 non-null    object 
 1   COM-1             236 non-null    object 
 2   COM-3             236 non-null    object 
 3   POP-4             236 non-null    int64  
 4   STA-1             236 non-null    object 
 5   STA-2             236 non-null    object 
 6   STA-4             236 non-null    int64  
 7   STA-5             236 non-null    object 
 8   STA-6             236 non-null    int64  
 9   STA-7             236 non-null    object 
 10  STA-10            236 non-null    object 
 11  STA-11            236 non-null    object 
 12  STA-12            236 non-null    object 
 13  TEC-1             236 non-null    int64  
 14  TEC-2.1           207 non-null    object 
 15  TEC-2.2           236 non-null    object 
 16  TEC-2.3           208 non-null    object 
 1